<a href="https://githubtocolab.com/gee-community/geemap/blob/master/examples/notebooks/33_accuracy_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [1]:
pip install geemap

  Using cached bqplot-0.12.43-py2.py3-none-any.whl.metadata (6.4 kB)
  Using cached colour-0.1.5-py2.py3-none-any.whl.metadata (18 kB)
  Using cached earthengine_api-0.1.411-py3-none-any.whl.metadata (1.8 kB)
  Using cached eerepr-0.0.4-py3-none-any.whl.metadata (4.0 kB)
  Using cached folium-0.17.0-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached geocoder-1.38.1-py2.py3-none-any.whl.metadata (14 kB)
  Using cached ipyevents-2.0.2-py3-none-any.whl.metadata (2.9 kB)
  Using cached ipyfilechooser-0.6.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached ipytree-0.2.2-py2.py3-none-any.whl.metadata (849 bytes)
  Using cached plotly-5.22.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached pyperclip-1.9.0-py3-none-any.whl
  Using cached pyshp-2.3.1-py2.py3-none-any.whl.metadata (55 kB)
  Using cached python_box-7.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.8 kB)
  Using cached scooby-0.10.0-py3-none-any.whl.metadata (15 kB)
  Using cached google_cloud_storag

# Machine Learning with Earth Engine - Accuracy Assessment

## Supervised classification algorithms available in Earth Engine

Source: https://developers.google.com/earth-engine/classification

The `Classifier` package handles supervised classification by traditional ML algorithms running in Earth Engine. These classifiers include CART, RandomForest, NaiveBayes and SVM. The general workflow for classification is:

1. Collect training data. Assemble features which have a property that stores the known class label and properties storing numeric values for the predictors.
2. Instantiate a classifier. Set its parameters if necessary.
3. Train the classifier using the training data.
4. Classify an image or feature collection.
5. Estimate classification error with independent validation data.

The training data is a `FeatureCollection` with a property storing the class label and properties storing predictor variables. Class labels should be consecutive, integers starting from 0. If necessary, use remap() to convert class values to consecutive integers. The predictors should be numeric.

To assess the accuracy of a classifier, use a `ConfusionMatrix`. The `sample()` method generates two random samples from the input data: one for training and one for validation. The training sample is used to train the classifier. You can get resubstitution accuracy on the training data from `classifier.confusionMatrix()`. To get validation accuracy, classify the validation data. This adds a `classification` property to the validation `FeatureCollection`. Call `errorMatrix()` on the classified `FeatureCollection` to get a confusion matrix representing validation (expected) accuracy.

![](https://i.imgur.com/vROsEiq.png)

## Step-by-step tutorial

### Import libraries

In [2]:
import ee
import geemap

### Create an interactive map

In [3]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Add data to the map

Let's add the [USGS National Land Cover Database](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD), which can be used to create training data with class labels. 

![](https://i.imgur.com/7QoRXxu.png)

In [4]:
NLCD2016 = ee.Image("USGS/NLCD/NLCD2016").select("landcover")
Map.addLayer(NLCD2016, {}, "NLCD 2016")

Load the NLCD metadata to find out the Landsat image IDs used to generate the land cover data.

In [5]:
NLCD_metadata = ee.FeatureCollection("users/giswqs/landcover/NLCD2016_metadata")
Map.addLayer(NLCD_metadata, {}, "NLCD Metadata")

In [6]:
# point = ee.Geometry.Point([-122.4439, 37.7538])  # Sanfrancisco, CA
# point = ee.Geometry.Point([-83.9293, 36.0526])   # Knoxville, TN
point = ee.Geometry.Point([-88.3070, 41.7471])  # Chicago, IL

In [7]:
metadata = NLCD_metadata.filterBounds(point).first()
region = metadata.geometry()

In [8]:
metadata.get("2016on_bas").getInfo()

'LC08_2016256'

In [9]:
doy = metadata.get("2016on_bas").getInfo().replace("LC08_", "")
doy

'2016256'

In [10]:
ee.Date.parse("YYYYDDD", doy).format("YYYY-MM-dd").getInfo()

'2016-09-12'

In [11]:
start_date = ee.Date.parse("YYYYDDD", doy)
end_date = start_date.advance(1, "day")

In [12]:
image = (
    ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
    .filterBounds(point)
    .filterDate(start_date, end_date)
    .first()
    .select("B[1-7]")
    .clip(region)
)

vis_params = {"min": 0, "max": 3000, "bands": ["B5", "B4", "B3"]}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
Map

/opt/conda/lib/python3.11/site-packages/ee/deprecation.py:204: DeprecationWarning: 

Attention required for LANDSAT/LC08/C01/T1_SR! You are using a deprecated asset.
To ensure continued functionality, please update it by July 1, 2024.
Learn more: https://developers.google.com/earth-engine/landsat_c1_to_c2

  warnings.warn(warning, category=DeprecationWarning)


Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

In [13]:
nlcd_raw = NLCD2016.clip(region)
Map.addLayer(nlcd_raw, {}, "NLCD")

### Prepare for consecutive class labels

In this example, we are going to use the [USGS National Land Cover Database (NLCD)](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD) to create label dataset for training.

First, we need to use the `remap()` function to turn class labels into consecutive integers.

In [14]:
raw_class_values = nlcd_raw.get("landcover_class_values").getInfo()
print(raw_class_values)

[11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 51, 52, 71, 72, 73, 74, 81, 82, 90, 95]


In [15]:
n_classes = len(raw_class_values)
new_class_values = list(range(0, n_classes))
new_class_values

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [16]:
class_palette = nlcd_raw.get("landcover_class_palette").getInfo()
print(class_palette)

['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [17]:
nlcd = nlcd_raw.remap(raw_class_values, new_class_values).select(
    ["remapped"], ["landcover"]
)
nlcd = nlcd.set("landcover_class_values", new_class_values)
nlcd = nlcd.set("landcover_class_palette", class_palette)

In [18]:
Map.addLayer(nlcd, {}, "NLCD")
Map

Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

### Make training data

In [19]:
# Make the training dataset.
points = nlcd.sample(
    **{
        "region": region,
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, "training", False)

In [20]:
print(points.size().getInfo())

5000


In [21]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-88.56212124700772, 42.210469414463425]}, 'id': '0', 'properties': {'landcover': 17}}


### Split training and testing

In [22]:
# Use these bands for prediction.
bands = ["B1", "B2", "B3", "B4", "B5", "B6", "B7"]

# This property of the table stores the land cover labels.
label = "landcover"

# Overlay the points on the imagery to get training.
sample = image.select(bands).sampleRegions(
    **{"collection": points, "properties": [label], "scale": 30}
)

# Adds a column of deterministic pseudorandom numbers.
sample = sample.randomColumn()

split = 0.7

training = sample.filter(ee.Filter.lt("random", split))
validation = sample.filter(ee.Filter.gte("random", split))

In [23]:
training.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'B1': 174,
  'B2': 223,
  'B3': 614,
  'B4': 368,
  'B5': 4221,
  'B6': 1737,
  'B7': 766,
  'landcover': 17,
  'random': 0.5253550035172192}}

In [24]:
validation.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '4_0',
 'properties': {'B1': 171,
  'B2': 201,
  'B3': 429,
  'B4': 301,
  'B5': 3478,
  'B6': 1698,
  'B7': 707,
  'landcover': 16,
  'random': 0.756636818947484}}

### Train the classifier

In this examples, we will use random forest classification.

In [25]:
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

### Classify the image

In [26]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, "classified")
Map

Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

### Render categorical map

To render a categorical map, we can set two image properties: `classification_class_values` and `classification_class_palette`. We can use the same style as the NLCD so that it is easy to compare the two maps.

In [27]:
class_values = nlcd.get("landcover_class_values").getInfo()
print(class_values)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [28]:
class_palette = nlcd.get("landcover_class_palette").getInfo()
print(class_palette)

['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [29]:
landcover = result.set("classification_class_values", class_values)
landcover = landcover.set("classification_class_palette", class_palette)

In [30]:
Map.addLayer(landcover, {}, "Land cover")
Map

Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

### Visualize the result

In [31]:
print("Change layer opacity:")
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

### Add a legend to the map

In [32]:
Map.add_legend(builtin_legend="NLCD")
Map

Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

### Accuracy assessment

#### Training dataset

`confusionMatrix()` computes a 2D confusion matrix for a classifier based on its training data (ie: resubstitution error). Axis 0 of the matrix correspond to the input classes (i.e., reference data), and axis 1 to the output classes (i.e., classification data). The rows and columns start at class 0 and increase sequentially up to the maximum class value

In [33]:
train_accuracy = classifier.confusionMatrix()

In [34]:
train_accuracy.getInfo()

[[293, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 0, 177, 6, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 2, 14, 0, 0],
 [1, 0, 1, 427, 4, 3, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 17, 0, 0],
 [0, 0, 0, 11, 181, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 2, 7, 94, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 2, 0, 0, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 5, 0, 0, 0, 187, 0, 0, 0, 0, 0, 0, 0, 0, 1, 8, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 34, 0, 0, 0, 0, 3, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Overall Accuracy essentially tells us out of all of the reference sites what proportion were mapped correctly. The overall accuracy is usually expressed as a percent, with 100% accuracy being a perfect classification where all reference site were classified correctly. Overall accuracy is the easiest to calculate and understand but ultimately only provides the map user and producer with basic accuracy information.

In [35]:
train_accuracy.accuracy().getInfo()

0.9569677970931889

The Kappa Coefficient is generated from a statistical test to evaluate the accuracy of a classification. Kappa essentially evaluates how well the classification performed as compared to just randomly assigning values, i.e. did the classification do better than random. The Kappa Coefficient can range from -1 t0 1. A value of 0 indicated that the classification is no better than a random classification. A negative number indicates the classification is significantly worse than random. A value close to 1 indicates that the classification is significantly better than random.

In [36]:
train_accuracy.kappa().getInfo()

0.9388448268519107

Producer's Accuracy is the map accuracy from the point of view of the map maker (the producer). This is how often are real features on the ground correctly shown on the classified map or the probability that a certain land cover of an area on the ground is classified as such. The Producer's Accuracy is complement of the Omission Error, Producer's Accuracy = 100%-Omission Error. It is also the number of reference sites classified accurately divided by the total number of reference sites for that class.

In [37]:
train_accuracy.producersAccuracy().getInfo()

[[1],
 [0],
 [0.8592233009708737],
 [0.9384615384615385],
 [0.9141414141414141],
 [0.912621359223301],
 [0.8823529411764706],
 [0.9211822660098522],
 [1],
 [0.9285714285714286],
 [0],
 [1],
 [0.8292682926829268],
 [0],
 [0],
 [0],
 [0.8571428571428571],
 [0.9937712344280861],
 [0.8266666666666667],
 [0.9047619047619048]]

The Consumer's Accuracy is the accuracy from the point of view of a map user, not the map maker. the User's accuracy essentially tells use how often the class on the map will actually be present on the ground. This is referred to as reliability. The User's Accuracy is complement of the Commission Error, User's Accuracy = 100%-Commission Error. The User's Accuracy is calculating by taking the total number of correct classifications for a particular class and dividing it by the row total.

In [38]:
train_accuracy.consumersAccuracy().getInfo()

[[0.979933110367893,
  0,
  0.9672131147540983,
  0.9143468950749465,
  0.9329896907216495,
  0.9215686274509803,
  0.9375,
  0.935,
  0.75,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0.9696969696969697,
  0.9674751929437707,
  0.9841269841269841,
  1]]

#### Validation dataset

In [39]:
validated = validation.classify(classifier)

In [40]:
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '4_0',
 'properties': {'B1': 171,
  'B2': 201,
  'B3': 429,
  'B4': 301,
  'B5': 3478,
  'B6': 1698,
  'B7': 707,
  'classification': 17,
  'landcover': 16,
  'random': 0.756636818947484}}

`errorMatrix` computes a 2D error matrix for a collection by comparing two columns of a collection: one containing the actual values, and one containing predicted values.

In [41]:
test_accuracy = validated.errorMatrix("landcover", "classification")

In [42]:
test_accuracy.getInfo()

[[123, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 22, 23, 2, 2, 0, 15, 0, 0, 0, 0, 0, 0, 0, 0, 4, 39, 7, 0],
 [0, 0, 8, 108, 15, 1, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 2, 35, 3, 0],
 [0, 0, 2, 27, 26, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0],
 [0, 0, 1, 2, 16, 21, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0],
 [0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 11, 7, 0, 0, 0, 59, 0, 0, 0, 0, 0, 0, 0, 0, 1, 21, 11, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 3, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [43]:
test_accuracy.accuracy().getInfo()

0.7069081153588196

In [44]:
test_accuracy.kappa().getInfo()

0.5713265011404898

In [45]:
test_accuracy.producersAccuracy().getInfo()

[[0.968503937007874],
 [0],
 [0.19298245614035087],
 [0.6101694915254238],
 [0.37142857142857144],
 [0.4772727272727273],
 [0],
 [0.5363636363636364],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.06521739130434782],
 [0.9162234042553191],
 [0.13636363636363635],
 [0]]

In [46]:
test_accuracy.consumersAccuracy().getInfo()

[[0.9919354838709677,
  0,
  0.3728813559322034,
  0.5192307692307693,
  0.4,
  0.4883720930232558,
  0,
  0.5175438596491229,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.16666666666666666,
  0.8311218335343787,
  0.11538461538461539,
  0]]

### Download confusion matrix

In [47]:
#import csv
#import os

#out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
#training_csv = os.path.join(out_dir, "train_accuracy.csv")
#testing_csv = os.path.join(out_dir, "test_accuracy.csv")

#with open(training_csv, "w", newline="") as f:
#    writer = csv.writer(f)
#    writer.writerows(train_accuracy.getInfo())

#with open(testing_csv, "w", newline="") as f:
#    writer = csv.writer(f)
#    writer.writerows(test_accuracy.getInfo())

In [54]:
import os
import csv

out_dir = "/home/jovyan/Downloads"
training_csv = os.path.join(out_dir, "train_accuracy.csv")
testing_csv = os.path.join(out_dir, "test_accuracy.csv")

# Create the directory if it doesn't exist
#os.makedirs(out_dir, exist_ok=True)

# Write to the training CSV file
with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(train_accuracy.getInfo())  # Ensure train_accuracy.getInfo() returns the data correctly

# Write to the testing CSV file
with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(test_accuracy.getInfo())  # Ensure test_accuracy.getInfo() returns the data correctly

### Reclassify land cover map

In [55]:
landcover = landcover.remap(new_class_values, raw_class_values).select(
    ["remapped"], ["classification"]
)

In [56]:
landcover = landcover.set("classification_class_values", raw_class_values)
landcover = landcover.set("classification_class_palette", class_palette)

In [57]:
Map.addLayer(landcover, {}, "Final land cover")
Map

Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

### Export the result

Export the result directly to your computer:

In [58]:
import os

out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
out_file = os.path.join(out_dir, "landcover.tif")

In [59]:
geemap.ee_export_image(landcover, filename=out_file, scale=900)

Generating URL ...
Please wait ...
Data downloaded to /home/jovyan/Downloads/landcover.tif


Export the result to Google Drive:

In [60]:
geemap.ee_export_image_to_drive(
    landcover, description="landcover", folder="export", scale=900
)